# Data Diagnostics

This notebook is to check the data quality for each subject, and to find out if some subjects have data that looks strange and we should look more into. 

What we are checking here:
- Min / Max / Mean values of relevant features (CGM and heartrate), compared to predefined expectations and thresholds
- Ratios between basal / bolus / carbs to check for unreasonable data
- Verify that time intervals are correct
- Feature sparsity in train / test data for each subject after imputation

Results storage and visualization:
- Saving a dataframe with results
- Plot results in color coded tables 

### Imports

In [1]:
import pandas as pd
import numpy as np
import os

### Load Data

In [12]:
folder = 'processed_data'
file_name = 'tidepool_dataset.csv'
df = pd.read_csv(os.path.join('..', folder, file_name), index_col='date', parse_dates=['date'], low_memory=False)

In [13]:
train_data = []
test_data = []

for subject_id, subset_df in df.groupby('id'): 
    def get_data_from_trimmed_data(df_subset, is_test=True):        
        # Validating time intervals
        time_diffs = df_subset.index.to_series().diff()
        expected_interval = pd.Timedelta(minutes=5)
        valid_intervals = (time_diffs[1:] == expected_interval).all()
        if not valid_intervals:
            invalid_intervals = time_diffs[time_diffs != expected_interval]
            print(f"Subject {subject_id} for is_test {is_test} has invalid intervals found:", invalid_intervals)
        
        cgm_col = 'CGM_smoothed' if 'CGM_smoothed' in df_subset.columns else 'CGM'
        
        # Trim data from nan values in the beginning and end
        first_valid_index = df_subset[cgm_col].first_valid_index()
        last_valid_index = df_subset[cgm_col].last_valid_index()
        if first_valid_index is not None and last_valid_index is not None:
            trimmed_subject_data = df_subset.loc[first_valid_index:last_valid_index]
        else:
            print(f'Subject {subject_id} for is_test {is_test} does not have valid data! DF is probably empty.')
            #return {'subject_id': subject_id}
            trimmed_subject_data = df_subset
            
        def safe_round(val):
            if pd.isna(val):  # Check if the value is NaN
                return val  # Return NaN as is
            else:
                return round(val)  # Round the value if it's not NaN
        daily_bolus = safe_round(trimmed_subject_data['bolus'].mean() * 12*24)
        daily_basal = safe_round(trimmed_subject_data['basal'].mean()/12 * 12*24)
        daily_carbs = safe_round(trimmed_subject_data['carbs'].mean() * 12*24)
        basal_ratio = safe_round(daily_basal / (daily_bolus + daily_basal) * 100)
        
        subject_summary = {
            'subject_id': subject_id,
            'mean_CGM': safe_round(trimmed_subject_data[cgm_col].mean()),
            'min_CGM': safe_round(trimmed_subject_data[cgm_col].min()),
            'max_CGM': safe_round(trimmed_subject_data[cgm_col].max()),
            'daily_basal_ratio': basal_ratio,
            'daily_bolus_ratio': 100 - basal_ratio,
            'daily_carbs_ratio': np.nan if daily_carbs == 0 else safe_round(daily_bolus / daily_carbs * 100),
            #'is_test': is_test,
        }
        if 'heartrate' in subset_df.columns:
            subject_summary['mean_heartrate'] = safe_round(trimmed_subject_data['heartrate'].mean())
            subject_summary['min_heartrate'] = safe_round(trimmed_subject_data['heartrate'].min())
            subject_summary['max_heartrate'] = safe_round(trimmed_subject_data['heartrate'].max())
        
        # Add sparsity of each feature
        percentages = []
        for col in trimmed_subject_data.columns:
            if not col in ['id', 'is_test']:
                nan_percentage = round(trimmed_subject_data[col].isna().mean() * 100, 1)
                if nan_percentage >= 100:
                    subject_summary[col] = np.nan
                    percentages.append(100)
                else:
                    subject_summary[col] = nan_percentage
                    percentages.append(nan_percentage)
        mean_value = np.mean(percentages)
        subject_summary['mean_feature_sparsity'] = round(mean_value, 1)
        return subject_summary
    
    train_data.append(get_data_from_trimmed_data(subset_df[subset_df['is_test'] == False], False))
    test_data.append(get_data_from_trimmed_data(subset_df[subset_df['is_test'] == True], True))    
    
train_summary_df = pd.DataFrame(train_data)
test_summary_df = pd.DataFrame(test_data)


In [4]:
train_summary_df

,subject_id,mean_CGM,min_CGM,max_CGM,daily_basal_ratio,daily_bolus_ratio,daily_carbs_ratio,CGM,carbs,bolus,basal,insulin,workout_label,calories_burned,mean_feature_sparsity
0,SAP100-07d808c00b707b2dc65962ebff546b731710451...,139,39,422,77,23,48.0,3.4,98.0,0.0,0.0,0.0,NaN,NaN,43.1
1,SAP100-07f024f9b2df04a3556a231aca535df04709564...,124,39,401,45,55,16.0,20.5,0.1,0.0,0.0,0.0,NaN,NaN,31.5
2,SAP100-0a1f3ac86f7620ee531a6131bdc7844f57b6f70...,161,39,421,48,52,23.0,7.2,0.3,0.2,0.2,0.2,NaN,NaN,29.7
3,SAP100-0ae3c54f576290d8c4b188adddac94c64b69e44...,128,39,396,50,50,117.0,2.0,0.2,0.1,0.1,0.1,NaN,NaN,28.9
4,SAP100-10005260d3ff0ed5ebefa82372e10d0616b0968...,173,39,432,37,63,22.0,10.5,0.1,0.1,0.1,0.1,NaN,NaN,30.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,SAP100-f945009b76d237c14445d6de9a9478f407d9e9c...,143,36,405,48,52,11.0,4.6,0.0,0.1,0.0,0.1,NaN,NaN,29.3
96,SAP100-fa6d14d94cca2cb2c33b9acd791a8fdbbc174c5...,136,39,401,42,58,467.0,6.7,0.6,0.0,0.0,0.0,NaN,NaN,29.6
97,SAP100-fc2b12608d03f91a22838fa8f7b7a03314c3071...,143,28,397,51,49,36.0,8.8,0.0,0.0,0.0,0.1,NaN,NaN,29.8
98,SAP100-fcb9778136ca5268cafc7e9cad4eab8c9bb15be...,127,39,401,48,52,7.0,7.1,0.3,0.3,0.2,0.3,NaN,NaN,29.7


### Create a Styled CSV for Feature Sparsity

In [5]:
def style_feature_sparsity(df_features):
    exclude_substrings = ['id', 'is_test', 'daily', 'mean', 'min', 'max']
    features = [col for col in train_summary_df.columns if not any(substring in col for substring in exclude_substrings)]
    
    # Function to convert RGB to Hex
    def rgb_to_hex(r, g, b):
        """Convert RGB to hex color."""
        return f'#{int(r):02x}{int(g):02x}{int(b):02x}'
    
    # Define styling function for color scale
    def highlight_severity(val):
        if pd.isna(val):  # Check if the value is NaN
            return 'background-color: white'  # White for NaN
        if val < 30:
            red, green, blue = 0, 255, 0  # Green for values less than 30
        elif val < 70:
            red, green, blue = 255, 255, 0  # Yellow for values less than 70
        else:
            red, green, blue = 255, 0, 0  # Red for values 70 and above
        hex_color = rgb_to_hex(red, green, blue)
        return f'background-color: {hex_color}'
    
    df_features = df_features.style.applymap(highlight_severity, subset=features)
    return df_features

## Check if CGM and Heartrate Values are Reasonable

In [6]:
def rgb_to_hex(r, g, b):
    """Convert RGB to hex color."""
    return f'#{int(r):02x}{int(g):02x}{int(b):02x}'

def highlight_range_severity(val, range_min, range_max):
    if pd.isna(val):
        return 'background-color: white'
    if range_min <= val <= range_max:
        red = 0  # No red component
        green = 255  # Full green
        blue = 0  # No blue component
    else:
        red = 255  # Full red
        green = 0  # No green component
        blue = 0  # No blue component
    hex_color = rgb_to_hex(red, green, blue)
    return f'background-color: {hex_color}'

def style_cgm_and_heartrate(df_features):
    df_features = df_features.applymap(lambda val: highlight_range_severity(val, 70, 220), subset=['mean_CGM'])
    df_features = df_features.applymap(lambda val: highlight_range_severity(val, 10, 100), subset=['min_CGM'])
    df_features = df_features.applymap(lambda val: highlight_range_severity(val, 200, 750), subset=['max_CGM'])
    
    if 'mean_heartrate' in df_features.columns:
        df_features = df_features.applymap(lambda val: highlight_range_severity(val, 30, 100), subset=['mean_heartrate'])
        df_features = df_features.applymap(lambda val: highlight_range_severity(val, 30, 100), subset=['min_heartrate'])
        df_features = df_features.applymap(lambda val: highlight_range_severity(val, 80, 250), subset=['max_heartrate'])
    
    return df_features

## Check if Insulin Carb Ratios are Reasonable

The reason for this check is to see if there are any weird values in bolus, basal or carbs. We use the ratio between those values to determine that. 

We consider that the ratio between bolus and basal should be from 40-60 to 70-30. 

Reasoning carb ratio: Imagine that 1U of insulin covers around 10-15 g of carbohydrates. If it is 10, there is a bolus-carbs relationship of 1:10. 

So if we let the gap be from 5 to 50% of "normal" ratios, we say that it is normal that one bolus dose covers everything from 5 to 50g of carbohydrates.

In [7]:
def style_insulin_carb_ratios(df_features):
    # We expect the basal-bolus ratio to be between 30-60% of total insulin, bolus to be around 40-70%
    # And bolus should be around 5-50% of carbs
    df_features = df_features.applymap(lambda val: highlight_range_severity(val, 30, 60), subset=['daily_basal_ratio'])
    df_features = df_features.applymap(lambda val: highlight_range_severity(val, 40, 70), subset=['daily_bolus_ratio'])
    df_features = df_features.applymap(lambda val: highlight_range_severity(val, 5, 50), subset=['daily_carbs_ratio'])
    return df_features

## Save the Color Coded Data 

In [8]:
def style_all_columns_and_save(df_features, is_test):
    df_features = style_feature_sparsity(df_features)
    df_features = style_cgm_and_heartrate(df_features)
    df_features = style_insulin_carb_ratios(df_features)
    
    if is_test:
        save_file_name = f'{file_name.split(".")[0]}_test.xlsx'
    else:
        save_file_name = f'{file_name.split(".")[0]}_train.xlsx'
    save_path = os.path.join('..', 'data_diagnostics', save_file_name)
    df_features.to_excel(save_path, engine='openpyxl', index=False)

#style_all_columns_and_save(train_summary_df, is_test=False)
#style_all_columns_and_save(test_summary_df, is_test=True)

In [9]:
# Sorting the ids with lower feature sparsity first
train_sorted_summary_df = train_summary_df.sort_values(by='mean_feature_sparsity', ascending=True)
test_sorted_summary_df = test_summary_df.sort_values(by='mean_feature_sparsity', ascending=True)


In [10]:
train_sorted_summary_df

,subject_id,mean_CGM,min_CGM,max_CGM,daily_basal_ratio,daily_bolus_ratio,daily_carbs_ratio,CGM,carbs,bolus,basal,insulin,workout_label,calories_burned,mean_feature_sparsity
34,SAP100-669338c27c9fde311943744a522cefc0b2b6d5a...,118,38,293,33,67,5.0,7.0,0.1,0.1,0.0,0.1,99.6,0.6,15.4
46,SAP100-83ce9ebcbeb53a3dfc0899d3bad758902b7f294...,119,39,401,66,34,21.0,7.1,0.0,0.0,0.0,0.0,98.1,10.0,16.5
83,SAP100-e09716a3c053b0bbfa9bdaf4befecae8edc81ff...,150,35,481,33,67,22.0,15.6,0.1,0.1,0.0,0.1,99.4,24.7,20.0
67,SAP100-baf43b5cdb97c109223bbde6063c6632a4781a2...,156,35,445,51,49,46.0,8.0,51.6,2.9,2.7,2.9,98.7,1.3,24.0
73,SAP100-cc5ecefec5975974e2362083806d04671ac3878...,105,39,257,46,54,3.0,2.1,0.0,0.2,0.2,0.2,99.2,75.8,25.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,SAP100-2868b266cbd84932c19b09551ea74432effd6d4...,126,39,401,45,55,15.0,2.7,99.8,0.1,0.0,0.1,NaN,NaN,43.2
80,SAP100-dbcb6083fae7a69dd4475687e85061031aaa1d1...,201,39,467,74,26,NaN,5.2,NaN,0.1,0.0,0.1,NaN,96.8,43.2
33,SAP100-65d013a5477dada4f1c10b7ea92befaeefd04c1...,174,39,467,59,41,0.0,2.7,NaN,0.1,0.0,0.1,NaN,NaN,43.3
47,SAP100-85605cc137c255c800d2c48d843900650a979a7...,153,39,458,58,42,0.0,2.8,NaN,0.0,0.0,0.0,NaN,NaN,43.3


In [11]:
style_all_columns_and_save(train_sorted_summary_df, is_test=False)
style_all_columns_and_save(test_sorted_summary_df, is_test=True)
